# Wrangle

In [1]:
#imports
import pandas as pd
import numpy as np

# system
import sys
# sys.path.append("../data")
sys.path.append("util_")
import _utility

In [2]:
# lacd data
df = pd.read_csv("data/known_exploited_vulnerabilities.csv")
df.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,NaN


In [3]:
# look at the shape
df.shape

(1140, 11)

In [4]:
#look at the columns
df.columns

Index(['cveID', 'vendorProject', 'product', 'vulnerabilityName', 'dateAdded',
       'shortDescription', 'requiredAction', 'dueDate',
       'knownRansomwareCampaignUse', 'notes', 'cwes'],
      dtype='object')

In [5]:
# check the full info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cveID                       1140 non-null   object 
 1   vendorProject               1140 non-null   object 
 2   product                     1140 non-null   object 
 3   vulnerabilityName           1140 non-null   object 
 4   dateAdded                   1140 non-null   object 
 5   shortDescription            1140 non-null   object 
 6   requiredAction              1140 non-null   object 
 7   dueDate                     1140 non-null   object 
 8   knownRansomwareCampaignUse  1140 non-null   object 
 9   notes                       373 non-null    object 
 10  cwes                        0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 98.1+ KB


In [6]:
# select numeric
len(df.select_dtypes("number").columns) # count of numeric columns

1

In [7]:
#select object columns
len(df.select_dtypes("object").columns) #count of object columns

10

In [8]:
df.describe()

,cwes
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


**What I see:**

- I have 1140 rows and 11columns
- 1 of the 11 columns are numric while 10 of them are string object colums
- the numeric column `cwes` is null
- the `notes` column has 373 non null rows
- descriptive statistics says nothing on the `cwes` empty column

In [9]:
# remove the non useul unique id and the emplt column
df = df.drop(columns=["cveID","cwes"])
df.columns

Index(['vendorProject', 'product', 'vulnerabilityName', 'dateAdded',
       'shortDescription', 'requiredAction', 'dueDate',
       'knownRansomwareCampaignUse', 'notes'],
      dtype='object')

In [10]:
# nanipulate the column names to a clear name
spaced_cols = []
col_renamed = ""
for name in df.columns:
    for ele in name:
        if ele == ele.upper():
            col_renamed += " " + ele
        else:
            col_renamed += ele
    spaced_cols.append(col_renamed)
    col_renamed = ""

df.columns = spaced_cols
spaced_cols

['vendor Project',
 'product',
 'vulnerability Name',
 'date Added',
 'short Description',
 'required Action',
 'due Date',
 'known Ransomware Campaign Use',
 'notes']

In [11]:
# remove spaces from the ends
# replace " " with _
# convert from lower case
new_cols = df.columns.str.strip().str.replace(" ", "_").str.lower()
df.columns = new_cols #replace the original columns with the new columns
df.columns

Index(['vendor_project', 'product', 'vulnerability_name', 'date_added',
       'short_description', 'required_action', 'due_date',
       'known_ransomware_campaign_use', 'notes'],
      dtype='object')

In [12]:
# remove duplicated rows
print("original data size: ", df.shape)
df = df.drop_duplicates(keep="first")
print("new data size: ", df.shape)

original data size:  (1140, 9)
new data size:  (1099, 9)


In [13]:
# Split the data into 3 sets
train, validate, test = _utility.split_data_(df=df, 
                     test_size=0.2, 
                     validate_size= 0.2, 
                     stratify_col="known_ransomware_campaign_use", 
                     random_state=10)
train.shape, validate.shape, test.shape

((659, 9), (220, 9), (220, 9))

In [14]:
_utility.save_original_data(df=train, folder="./data", file_name="temp_train_data_to_be_removed")

'File temp_train_data_to_be_removed saved'

In [15]:
# reset index
train = train.reset_index(drop=True)

In [16]:
# count values for each feature
for col in train.columns:
    print(train[col].value_counts)
    print("unique value count size:",train[col].value_counts().shape)
    print("\n\n")

<bound method IndexOpsMixin.value_counts of 0          Zimbra
1          Google
2      SolarWinds
3         Mozilla
4           Rails
          ...    
654         Adobe
655     Hikvision
656     Microsoft
657         Cisco
658        Google
Name: vendor_project, Length: 659, dtype: object>
unique value count size: (121,)



<bound method IndexOpsMixin.value_counts of 0                         Collaboration (ZCS)
1                                 Chromium V8
2                                      Serv-U
3                     Firefox and Thunderbird
4                               Ruby on Rails
                        ...                  
654                                ColdFusion
655               Security cameras web server
656                         Internet Explorer
657    IOS Software and Cisco IOS XE Software
658                               Chromium V8
Name: product, Length: 659, dtype: object>
unique value count size: (308,)



<bound method IndexOpsMixin.value_counts of 0

# Tasks from EDA

* The source of memory corruption is from 4 vendors, but it looks like it's happening on different products of these vendors, so I will go back inthe wrangle file and separate the product from the volunerability and create a additional column for specific vulnerabilities groupd by wording. (cell 7 - cell 15)

In [17]:
train_temp = train.copy()

In [18]:
# # Privilege Escalation Vulnerability  
# search_terms = ["Privilege", "Escalation"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [19]:
# # Memory Corruption Vulnerability
# search_terms = ["Memory", "Corruption"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [20]:
# # Arbitrary File Upload Vulnerability
# search_terms = ["File", "Upload"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [21]:
# # Remote/Arbitrary Code Execution Vulnerability 
# search_terms = ["Execution", "Remote", "Arbitrary"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [22]:
# # Improper Access Control Vulnerability
# search_terms = ["Access", "Improper", "Control"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [23]:
# # Command Injection Vulnerability
# search_terms = ["Injection", "Command", "SQL"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [24]:
# # Use-After-Free Vulnerability 
# search_terms = ["Use-After", "Use-After-Free"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [25]:
# # Deserialization of Untrusted Data Vulnerability
# search_terms = ["Deserialization", "Untrusted"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [26]:
# # Type Confusion Vulnerability  
# search_terms = ["Type Confusion"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [27]:
# # Path Traversal Vulnerability
# search_terms = ["Traversal", "path"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [28]:
# # Buffer Overflow Vulnerability
# search_terms = ["Buffer", "Overflow", "Heap"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [29]:
# # Denial of Service Vulnerability 
# search_terms = ["Denial", "Service"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [30]:
# # Information Disclosure Vulnerability
# search_terms = ["Information", "Disclosure"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [31]:
# # Out-of-Bounds Write Vulnerability
# search_terms = ["Out-of-Bounds", "Write", "Bounds"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [32]:
# # Sandbox Bypass Vulnerability 
# search_terms = ["Sandbox", "Bypass"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [33]:
# # Arbitrary File Upload Vulnerability
# search_terms = ["Unspecified"]
# pattern = '|'.join(search_terms)

# train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)]

#### I cam combine all of the obove conditions into one as I engineer a new feature for vulnerabilities

In [34]:
# Create a function that handles feature engineering for vulnerability names
def feature_eng_vul_names(data_frame):
    # Define the conditions
    conditions = [
        data_frame['vulnerability_name'].str.contains(r'Privilege|Escalation', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Memory|Corruption', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'File|Upload', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Remote Code', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Access|Improper|Control', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Injection|Command|SQL', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Use-After|Use-After-Free', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Deserialization|Untrusted', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Type Confusion', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Traversal|path', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Buffer|Overflow|Heap', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Denial|Service', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Information|Disclosure', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Out-of-Bounds|Write|Bounds', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Sandbox|Bypass', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Unspecified', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Spoofing', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Input|Validation', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Hard-Coded|Credentials', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Forgery|SSRF|Request|Server', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Scripting|XSS|Cross|Site', case=False, na=False),
        data_frame['vulnerability_name'].str.contains(r'Arbitrary Code', case=False, na=False)
    ]

    # Define the corresponding values for the new feature
    choices = [
        "Privilege Escalation Vulnerability",
        "Memory Corruption Vulnerability",
        "File Upload Vulnerability",
        "Remote Code Execution Vulnerability",
        "Improper Access Control Vulnerability",
        "Command Injection Vulnerability",
        "Use-After-Free Vulnerability",
        "Deserialization of Untrusted Data Vulnerability",
        "Type Confusion Vulnerability",
        "Path Traversal Vulnerability",
        "Buffer Overflow Vulnerability",
        "Denial of Service Vulnerability",
        "Information Disclosure Vulnerability",
        "Out-of-Bounds Write Vulnerability",
        "Sandbox Bypass Vulnerability",
        "Arbitrary File Upload Vulnerability",
        "Spoofing Vulnerabilities",
        "Input Validation Vulnerabilities",
        "Use of Hard-Coded Credentials Vulnerabilities",
        "Server-Side Request Forgery (SSRF) Vulnerabilities",
        "Cross-Site Scripting (XSS) Vulnerabilities",
        "Arbitrary Code Execution Vulnerability",
    ]

    # Use np.select to create the new feature
    data_frame['vulnerability_type'] = np.select(conditions, choices, default="Other Vulnerability")
    return data_frame


#### The newly engineered feature values

In [35]:
feature_eng_vul_names(train).vulnerability_type.value_counts()

vulnerability_type
Privilege Escalation Vulnerability                    93
Remote Code Execution Vulnerability                   90
Command Injection Vulnerability                       59
Improper Access Control Vulnerability                 46
Sandbox Bypass Vulnerability                          43
Memory Corruption Vulnerability                       39
Buffer Overflow Vulnerability                         39
Use-After-Free Vulnerability                          35
Other Vulnerability                                   34
Path Traversal Vulnerability                          24
Information Disclosure Vulnerability                  23
File Upload Vulnerability                             20
Out-of-Bounds Write Vulnerability                     18
Denial of Service Vulnerability                       18
Deserialization of Untrusted Data Vulnerability       16
Type Confusion Vulnerability                          15
Arbitrary File Upload Vulnerability                   12
Cross-Site S

**Let know look how we can engineer the `required_action` column**

In [36]:
train.required_action.value_counts()

required_action
Apply updates per vendor instructions.                                                                                                                                                                                                                                                                                                                                               505
Apply mitigations per vendor instructions or discontinue use of the product if mitigations are unavailable.                                                                                                                                                                                                                                                                           93
The impacted product is end-of-life and should be disconnected if still in use.                                                                                                                                                       

I see that there is only 15 options that represent the entire training dataset, howerver they are too wordy and I don;t like that, so I will engineer a new column withshort and specific identifiers for each required action .

In [37]:
# Create a function that handles feature engineering for vulnerability names
def feature_eng_req_actions(data_frame):
#     Create an array to contain the required actions
    actions = np.array(data_frame.required_action.value_counts().keys())
    # Define the conditions
    conditions = [
        data_frame['required_action'] == actions[0],
        data_frame['required_action'] == actions[1],
        data_frame['required_action'] == actions[2],
        data_frame['required_action'] == actions[3],
        data_frame['required_action'] == actions[4],
        data_frame['required_action'] == actions[5],
        data_frame['required_action'] == actions[6],
        data_frame['required_action'] == actions[7],
        data_frame['required_action'] == actions[8],
        data_frame['required_action'] == actions[9],
        data_frame['required_action'] == actions[10],
        data_frame['required_action'] == actions[11],
        data_frame['required_action'] == actions[12],
        data_frame['required_action'] == actions[13],
        data_frame['required_action'] == actions[14]
    ]

    # Define the corresponding values for the new feature
    choices = [
        "Apply Updates", 
        "Apply Mitigations or Discontinue", 
        "End of Life Disconnect",  
        "Apply Updates or Discontinue",  
        "Apply Remediations or Discontinue",  
        "Apply Updates or Discontinue",  
        "Legacy Disconnect",  
        "Verify Cisco Compliance",  
        "Mitigate GroupLock or Discontinue",  
        "End of Life Disconnect",  
        "Mitigate or Disable SLP",  
        "Log4j Remediation",  
        "Internet Block and Update",  
        "Enable Threat Prevention",  
        "DLink Update or Disconnect",
    ]

    # Use np.select to create the new feature
    data_frame['required_action_new'] = np.select(conditions, choices)
    return data_frame


In [38]:
feature_eng_req_actions(train).required_action_new.value_counts()

required_action_new
Apply Updates                        505
Apply Mitigations or Discontinue      93
End of Life Disconnect                32
Apply Updates or Discontinue          12
Apply Remediations or Discontinue      6
Legacy Disconnect                      3
Verify Cisco Compliance                2
Mitigate GroupLock or Discontinue      1
Mitigate or Disable SLP                1
Log4j Remediation                      1
Internet Block and Update              1
Enable Threat Prevention               1
DLink Update or Disconnect             1
Name: count, dtype: int64

**Now lets look into values contained in products**

In [39]:
product_groups= {
    "Operating Systems": [
        "XP", "OS X", "Windows", "Solaris", "IOS, IOS XR, and IOS XE", 
        "Android Kernel", "SFOS", "vCenter Server and Cloud Foundation", 
        "MacOS", "Linux", "iOS", "iOS, iPadOS, and watchOS", 
        "iOS, iPadOS, and macOS", "iOS and macOS", "iOS and iPadOS", 
        "RouterOS", "Unraid", "IOS Software", "IOS and IOS XE",
        'macOS', 'IOS software', 'Android OS', 'iOS, macOS, watchOS', 'Windows',
        'Solaris and Zettabyte File System (ZFS)', 'TerraMaster OS', 'Debian-specific Redis Servers',
        "Windows", "IOS", "IOS XE Software", "IOS, XR, and XE Software",
        "SonicOS", "Windows CNG Key Isolation Service", "Client-Server Run-time Subsystem (CSRSS)"
    ],
    "Microsoft Products": [
        "Defender", ".NET Core and Visual Studio", "DWM Core Library", 
        "Enhanced Cryptographic Provider", "Edge and Internet Explorer", 
        "Desktop", "SMBv1", ".NET Framework, SharePoint, Visual Studio", 
        "Task Scheduler", "Windows COM+ Event System Service", 
        "Internet Information Services (IIS)", "MiCollab, MiVoice Business Express", 
        "WinVerifyTrust function", "Malware Protection Engine", 
        "Forefront Threat Management Gateway (TMG)", 
        "Ancillary Function Driver (afd.sys)", "PowerPoint", "Excel", 
        "SharePoint", "Solution Manager", "Remote Desktop Services", 
        "Internet Explorer Scripting Engine", 
        "Active Management Technology (AMT), Small Business Technology (SBT), and Standard Manageability",
        "Office", "Exchange Server", "Win32k", "Word", "MSHTML", 
        "Active Directory", "Silverlight", "Graphics Device Interface (GDI)",
        '.NET Framework', 'SharePoint Server', 'Internet Explorer and Edge', 'OfficeScan',
        'Exchange', 'Office and WordPad', 'MSCOMCTL.OCX', 'Outlook',
        "WordPad", "Skype for Business", "XML Core Services"
    ],
    "Browsers": [
        "Chrome FreeType", "Chromium WebP", "Firefox, Firefox ESR, and Thunderbird", 
        "Chromium Intents", "Chromium Visuals", "Chromium Animation", 
        "Chrome WebAudio", "Chromium WebGL", "WebRTC", 
        "Chromium WebRTC", "Chromium Portals", "Chromium Skia", 
        "Chromium Mojo", "Internet Explorer", "Chromium V8", 
        "Firefox and Thunderbird", "Firefox", "Edge", 
        "Chromium", "Chromium Network Service",
        'Chrome', 'Chromium PopupBlocker', 'Chromium GPU', 'WebKitGTK', 'Chrome Blink',
        'Chromium libvpx', 'Chromium Indexed DB API', 'Chrome Media',
        "Toolbox Browser", "Chrome for Android UI", "Chromium Blink"
    ],
    "Adobe Products": [
        "Reader", "Flash Player", "Shockwave Player", "Photoshop", "Illustrator", 
        "Flash Player and AIR", "Adobe Reader", "Acrobat and Reader", 
        "Reader and Acrobat", "ColdFusion",
        'Acrobat and Reader, Flash Player'
    ],
    "Networking": [
        "DIR-600 Router", "Small Business RV160, RV260, RV340, and RV345 Series Routers", 
        "Small Business RV320 and RV325 Routers", "Ethernet Diagnostics Driver for Windows", 
        "EMG2926 Routers", "DWL-2600AP Access Point", "WNR2000v5 Router", 
        "DSL-2760U", "Wireless Access Point (WAP) Devices", 
        "RV Series Routers", "P660HN-T1A Routers", "DIR-859 Router", 
        "DIR-825 R1 Devices", "Multiple Archer Devices", "Multiple Routers", 
        "EyesOfNetwork", "Prime Data Center Network Manager (DCNM)", 
        "Aria Operations for Networks", "SD-WAN Edge", 
        "Service Location Protocol (SLP)", "Firewall", 
        "Secure Remote Access (SRA)", "Multiple Vigor Routers", 
        "DSL-2750B Devices", "Archer AX21", "Networking Hardware", 
        "Junos OS", "Multiple Chipsets", "IOS and IOS XE Software", 
        "IOS XR", "NetScaler ADC and NetScaler Gateway", 
        "Adaptive Security Appliance (ASA)", "Adaptive Security Appliance (ASA) and Firepower Threat Defense (FTD)", 
        "SD-WAN and NetScaler", "BIG-IP and BIG-IQ Centralized Management",
        "ProCurve Manager (PCM), PCM+, Identity Driven Manager (IDM), and Application Lifecycle Management",
        "VPN Routers", "Multiple NAS Devices", "Multiple Devices",
        "BIG-IP Configuration Utility", "ScreenConnect", "Connect Secure, Policy Secure, and Neurons","Secure Remote Access (SRA)",
        'Gigabit Passive Optical Network (GPON) Routers', 'DIR-300 Router', 'DIR-820L', 'DIR-605 Router',
        'DIR-610 Devices', 'Small Business RV320 and RV325 Dual Gigabit WAN VPN Routers', 'WARP, IPVPN, and MPVPN software',
        'AirOS', 'IPv6', 'Netlogon', 'DNS-320 Device', 'Cyber Infrastructure (ACI)',
        "Webmail", "Roundcube Webmail", "Serv-U", "BIG-IP", "Nagios XI", "Cacti", 
        "Samba", "HTTP File Server", "HTTP Protocol Stack", "rConfig", "Mail Transfer Agent (MTA)",
        "OpenSMTPD", "Aviatrix Controller", "OpenView Network Node Manager", "DIR-645 Router",
        "DGN2200 Devices", "WF2419 Devices", "Wireless Router DGN2200", "AC7, AC9, and AC10 Routers",
        "AC1900 Router AC15 Model", "AC11 Router", "Enhanced Multimedia Router (EMR)",
        "JGS516PE Devices", "Linear eMerge E3-Series", "Firebox and XTM"
    ],
    "Cisco Products": [
        "IOS XE Web UI", "NX-OS", "NetScaler SD-WAN Enterprise, CloudBridge Virtual WAN, and XenMobile Server", 
        "Adaptive Security Appliance and Firepower Threat Defense", "HyperFlex HX", 
        "UCM6200", "IOS and IOS XE Software", "IOS Software and Cisco IOS XE Software", 
        "IOS XR", "Adaptive Security Appliance (ASA)", 
        "Adaptive Security Appliance (ASA) and Firepower Threat Defense (FTD)", 
        "AnyConnect Secure", 'Cisco IOS XE Web UI', 'Cisco IP Phones', 'Catalyst 6800 Series Switches', 
        'Catalyst 4500 Series Switches and Cisco Catalyst 4500-X Series Switches','SMBv1 server', 'SMBv3',
        "IOS XE Software", "IOS, XR, and XE Software"
    ],
    "Security": [
        "Apex One and OfficeScan", "Endpoint Manager Mobile (EPMM)", 
        "Orion", "Symantec Messaging Gateway", "SG UTM", 
        "Application Delivery Controller (ADC), Gateway, and SD-WAN WANOP Appliance", 
        "SSLVPN SMA100", "Multiple Firewalls", "SMA 100 Appliances", 
        "Firebox and XTM Appliances", "Pulse Connect Secure and Pulse Policy Secure", 
        "SysAid Server", "Operation Bridge Reporter (OBR)", 
        "Unified Data Protection (UDP)", "FortiOS and FortiADC", 
        "FortiOS and FortiProxy", "McAfee Total Protection (MTP)", 
        "Security cameras web server", "Auditor", "VioStor NVR", "Sentry",
        "FortiOS", "Pulse Connect Secure", "PAN-OS", "Collaboration (ZCS)",
        "Apex One, Apex One as a Service, and Worry-Free Business Security",
        "Cobalt Strike", "SonicWall Email Security", "Crowd and Crowd Data Center",
        "Endpoint Manager Mobile (EPMM) and MobileIron Core", "MOVEit Transfer",
        "Kerberos Key Distribution Center (KDC)", "Backup & Replication", 
        "Polkit", "GoAnywhere MFT",'FortiClient EMS', 'FortiOS and FortiProxy SSL-VPN', 
        'Quantum Security Gateways', 'Email Security Gateway (ESG) Appliance',
        'SmartScreen Prompt', 'Secure Access Control System (ACS)', 'Script Security Plugin',
        "Open Management Infrastructure (OMI)", "Connect Secure and Policy Secure",
        "SMA100", "Application Delivery Controller (ADC) and Gateway", "Apex One and Apex One as a Service",
        "Apex One and Worry-Free Business Security", "Apex One, OfficeScan, and Worry-Free Business Security",
        "Apex One, OfficeScan and Worry-Free Business Security Agents", "KACE System Management Appliance",
        "Micro Focus Access Manager", "Authy", "MobileIron Multiple Products", "ManageEngine ServiceDesk Plus (SDP) / SupportCenter Plus",
        "ManageEngine ServiceDesk Plus (SDP)", "Social Warfare Plugin", "Snap Creek Duplicator Plugin",
        "Firebox and XTM", "Snapdragon Auto, Snapdragon Compute, Snapdragon Connectivity, Snapdragon Consumer IOT, Snapdragon Industrial IOT, Snapdragon Mobile, Snapdragon Voice & Music, Snapdragon Wearables",
        "Exim Internet Mailer", "DirectX Graphics Kernel (DXGKRNL)", "Access Management (AM)"
    ],
    "Development and Frameworks": [
        "October CMS", "Spring Cloud", "Spring Cloud Gateway", "Spring Framework", 
        "Sudo", "ImageMagick", "dotCMS", "PHPUnit", "GeoServer", "RocketMQ", 
        "Java Runtime Environment (JRE)", "Matrix Project Plugin", 
        "Frontend", "User Interface (UI) for ASP.NET AJAX", "Ruby on Rails", 
        "Telerik UI for ASP.NET AJAX", "Airflow", "Airflow's Experimental API", 
        "GitLab CE/EE", "Framework", "Confluence", "noneCms", 
        "PHP", "Java SE", "Struts", "NetWeaver", "Salt", "Log4j2", 
        "Jenkins Stapler Web Framework", "DotNetNuke (DNN)", "Struts 1", "ActiveMQ",
        'Jenkins User Interface (UI)', 'Java SE and JRockit', 
        'Joomla!', 'Laravel Framework', 'Spring Cloud Configuration (Config) Server',
        'BlazeDS', 'Apache', 'Struts 2', 'ChakraCore scripting engine', 'Symfony', 'ThinkPHP', 'GoAhead',
        "Spring Data Commons", "Telerik Report Server", "ASP.NET AJAX and Sitefinity", "Ignition",
        "Jungle Software Development Kit (SDK)", "Java SE JDK and JRE", "SDK", "JAI-EXT", "Shiro",
        "Superset", "Primefaces Application", "mongo-express", "AdminLTE", "Fuel CMS",
        "System Information Library for Node.JS", "AP-Router SDK", "BIG-IP Traffic Management Microkernel",
        "BIG-IP", "VTScada (formerly VTS)", "Web Studio", "OmniPCX Enterprise", "Novi Survey", 
        "Checkbox Survey", "PlaySMS", "Vision PLC and HMI", "Tools", "Trusted Firmware", "GNU C Library",
        "Web Appliance", "Apex Central", "Ignition", "Apex One", "Worry-Free Business Security",
        "DBUTIL Driver"
    ],
    "Web Servers and Middleware": [
        "HTTP Server", "Exim", "Openfire", "FastCGI Process Manager (FPM)", 
        "Internet Information Services (IIS)", "WS_FTP Server", 
        "Streaming Service Proxy", "Big-IP", "XStream", "HTTP.sys", 
        "Webmin", "Collaboration Suite (ZCS)", "HTTP File Server (HFS)", 
        "Content Collaboration", "WebRTC", "WebLogic Server", "Tomcat", 
        "Fusion Middleware", "WebSphere Application Server and Server Hypervisor Edition", 
        "Solr", "Bourne-Again Shell (Bash)",
        'JBoss', 'Confluence Data Center and Server', 'Elasticsearch', 'vBulletin', 'APISIX', 'Grafana',
        'JasperReports', 'Control Web Panel', 'Jenkins User Interface (UI)', 'Big-IP Traffic Management Microkernel',
        'Mirth Connect', 'Apache', 'Flink', 'Jenkins', 'JBoss Application Server', 'JBoss Seam 2',
        'Joomla!', 'Jira Server and Data Center', 'Drupal Core', 'Liferay Portal', 'Zimbra Collaboration (ZCS)',
        "Confluence Server and Data Center", "Confluence Server/Data Center", "Confluence Server",
        "Drupal core", "Ghostscript", "GLPI", "E-Business Suite", "Aspera Faspex", "StoreFront Server",
        "Media Server", "Workspace Application and Receiver for Windows", "JBoss RichFaces Framework",
        "BI Publisher (Formerly XML Publisher)", "UnRAR", "Archive_Tar", "Streaming Service",
        "Planning Analytics"
    ],
    "Virtualization and Cloud": [
        "ESXi", "MinIO", "Xen", "VirtualBox", "VMware", "KVM", 
        "Virtualization Manager", "Hyper-V RemoteFX", "Kylin", 
        "Workspace ONE Access and Identity Manager", 
        "vCenter Server", "Virtual System/Server Administrator (VSA)", 
        "ownCloud graphapi", 'VMware ESXi and Horizon DaaS',
        "Endpoint Manager Cloud Service Appliance (EPM CSA)", "CrushFTP", "Cloud", "Confluence Server and Data Server",
        "Cameras and Network Video Recorder (NVR)", "Authy"
    ],
    "Database": [
        "CouchDB", "Cassandra", "Redis", "MySQL", "PostgreSQL", 
        "MariaDB", "MongoDB", "Oracle", "InfoSphere BigInsights", 
        "Nexus Repository Manager", "Nexus Repository",
        "Commerce and Magento Open Source", "ManageEngine", "Data Risk Manager",
        'phpMyAdmin', 'Redis Servers', 'Mongo Express',
        "dbutil Driver", "Planning Analytics", "BI Publisher (Formerly XML Publisher)",
        "Spreadsheet::ParseExcel"
    ],
    "Storage": [
        "QNAP Network-Attached Storage (NAS)", "NVMS-1000", 
        "SureLine", "Synology NAS", "Seagate Drives", "DNS-320 Storage Device",
        "MiVoice Connect", "Network Attached Storage (NAS)", 
        "Backup Exec Agent", "Multiple Network-Attached Storage (NAS) Devices",
        'N1A1 NAS', 'Buffalo Firmware', 'QTS', 'vRealize Operations Manager API',
        "Samba", "Photo Station", "File Manager Plugin"
    ],
    "Hardware": [
        "Mali Graphics Processing Unit (GPU)", "Mali GPU Kernel Driver", "Pixel", "Mobile Devices",
        "Cameras and Network Video Recorder (NVR)", "SIMATIC CP", "DCS-930L Devices", "Compact", 
        "ATM Font Driver", "AE1021, AE1021PE", "ACDB Audio Driver"
    ]
}


In [40]:
def map_product_groups(product):
    for key, value in product_groups.items():
        if product in value:
            return key
    return "Other"
train["product_category"] = train["product"].apply(map_product_groups)
train.product_category.value_counts()

product_category
Operating Systems             117
Networking                     81
Microsoft Products             68
Other                          67
Security                       63
Development and Frameworks     58
Browsers                       56
Adobe Products                 43
Web Servers and Middleware     37
Virtualization and Cloud       17
Hardware                       15
Storage                        15
Cisco Products                 12
Database                       10
Name: count, dtype: int64

**Apply new changes to the entire dataframe**

In [41]:
df = feature_eng_vul_names(df)
df = feature_eng_req_actions(df)
df["product_category"] = df["product"].apply(map_product_groups)
df.head()

,vendor_project,product,vulnerability_name,date_added,short_description,required_action,due_date,known_ransomware_campaign_use,notes,vulnerability_type,required_action_new,product_category
0,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability,Apply Updates,Other
1,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability,Apply Updates,Other
2,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Command Injection Vulnerability,Apply Updates,Other
3,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,Server-Side Request Forgery (SSRF) Vulnerabili...,Apply Updates,Other
4,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,Buffer Overflow Vulnerability,Apply Updates,Adobe Products


In [42]:
# Create a month and uear column from the training date added
date_added = pd.to_datetime(df.date_added)
year = date_added.dt.year
month = date_added.dt.month
df["year_added"] = pd.Series(year).astype("int")
df["monthe_added"] = pd.Series(month).astype("int")

In [43]:
# remove columns not needed
df = df.drop(columns=["notes", "short_description", "vulnerability_name","date_added", "due_date", "required_action", "product"])
df.head(3)

,vendor_project,known_ransomware_campaign_use,vulnerability_type,required_action_new,product_category,year_added,monthe_added
0,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11
1,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11
2,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11


### Dummies

Create dummy variables for modeling

In [45]:

# First create a copy of the original dataframe
df_dummy = df.copy()

In [48]:
# get all the columns to upply dummies to
dummy_col = df_dummy.columns[df_dummy.columns != "known_ransomware_campaign_use"]
dummy_col = dummy_col[dummy_col != "date_added"]
df_dummy[dummy_col].head(3)

,vendor_project,vulnerability_type,required_action_new,product_category,year_added,monthe_added
0,Accellion,Command Injection Vulnerability,Apply Updates,Other,2021,11
1,Accellion,Command Injection Vulnerability,Apply Updates,Other,2021,11
2,Accellion,Command Injection Vulnerability,Apply Updates,Other,2021,11


In [49]:
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1099 entries, 0 to 1139
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   vendor_project                 1099 non-null   object
 1   known_ransomware_campaign_use  1099 non-null   object
 2   vulnerability_type             1099 non-null   object
 3   required_action_new            1099 non-null   object
 4   product_category               1099 non-null   object
 5   year_added                     1099 non-null   int64 
 6   monthe_added                   1099 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 68.7+ KB


In [50]:
# Generate dummy variables for modeling
df_dummy = pd.get_dummies(df_dummy[dummy_col], drop_first=True).astype(int)
df_dummy.head(3)

,year_added,monthe_added,vendor_project_Acronis,vendor_project_Adobe,vendor_project_Alcatel,vendor_project_Amcrest,vendor_project_Android,vendor_project_Apache,vendor_project_Apple,vendor_project_Arcadyan,...,product_category_Development and Frameworks,product_category_Hardware,product_category_Microsoft Products,product_category_Networking,product_category_Operating Systems,product_category_Other,product_category_Security,product_category_Storage,product_category_Virtualization and Cloud,product_category_Web Servers and Middleware
0,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


I have generated 230 new features for modeling.

**Make the targer binary**

* All `known` ransomware are labled as `1`, and all `unknown` copains are labeled with `0`.

In [48]:
df_dummy["known_ransomware_campaign_use"] = np.where(df.known_ransomware_campaign_use == "Known", 1, 0)
df_dummy.head(3)

,year_added,monthe_added,vendor_project_Acronis,vendor_project_Adobe,vendor_project_Alcatel,vendor_project_Amcrest,vendor_project_Android,vendor_project_Apache,vendor_project_Apple,vendor_project_Arcadyan,...,product_category_Hardware,product_category_Microsoft Products,product_category_Networking,product_category_Operating Systems,product_category_Other,product_category_Security,product_category_Storage,product_category_Virtualization and Cloud,product_category_Web Servers and Middleware,known_ransomware_campaign_use
0,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2021,11,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


## Split and Save all the data sets

In [49]:
# Split the data into 3 sets
train, validate, test = _utility.split_data_(df=df, 
                     test_size=0.2, 
                     validate_size= 0.2, 
#                      stratify_col="", 
                     random_state=10)
train.shape, validate.shape, test.shape

((659, 7), (220, 7), (220, 7))

In [50]:
df.head(3)

,vendor_project,known_ransomware_campaign_use,vulnerability_type,required_action_new,product_category,year_added,monthe_added
0,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11
1,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11
2,Accellion,Known,Command Injection Vulnerability,Apply Updates,Other,2021,11


In [51]:
# Split the data into 3 sets
train_dummy, validate_dummy, test_dummy = _utility.split_data_(df=df_dummy, 
                     test_size=0.2, 
                     validate_size= 0.2, 
#                      stratify_col="", 
                     random_state=10)
train_dummy.shape, validate_dummy.shape, test_dummy.shape

((659, 232), (220, 232), (220, 232))

In [52]:
# Save data sets
_utility.save_split_data_(original_df=df,
                         encoded_scaled_df= df_dummy,
                         train = train_dummy,
                          validate = validate_dummy,
                          test= test_dummy,
                          folder_path = "./data",
                          test_size= 0.2,
#                           stratify_col = "",
                          random_state= 10
                         )

'SIX data sets saved as .csv'

### What I found and What I did

**During Acquisition**

- I have 1140 rows and 11 columns.
- 1 of the 11 columns are numric, 1 is a datetime object and the other 9 of them are string object colums.
- The numeric column is `cwes`, but it is null.
- The `notes` column has 373 non null rows out of 1140 rows
- Descriptive statistics says nothing on the `cwes` empty column

**During preparation**

The following changes where applied to the entire data set

- I engineeredd thee new features `vulnerability_type`, `required_action_new`, and `product_category`. These thee features where generated by mapping features from the original data `vulnerability_name`, `required_action`, and `product`. This was done by categorizing all alike vulnerabilities, actions to take based on vulnerability, and the type of product. This helped condece the moltitude of descrate categories into digestable and visualy apealing categories.
- I created dummy variables by encoding all my categorical variables into `0's` and `1's`. For the target feature `known_ransomware_campaign_use` I encoded the `0` to `Unkown` ransomware campains, and `1` to `Known` ransomware campains.
- I removed all features that where null and features that added no value to the data,`notes, short_description, vulnerability_name, due_date, required_action, product`. Included to these features are the original en-engineered three features that lost purpose in the data.
- I finaly split my data into three sets `train`, `validate`, and `test` datasets with a `60/20/20` split with a random state of `10` and I avoided stratification at the target feature `known_ransomware_campaign_use` to avoid ddata leakage an get the most reliable data posible.
- I am saving 6 data sets in the `./data` folder as `.csv`'s. They include the original data, the prepared data with and with no dummies, the traing, calidation, and testing splits with dummies.